# Hyperbolical Embedding of CATH Labels and Protein Sequences

Hierarchy:
[CATH Labels Hierarchy](http://www.cathdb.info/browse/tree)

Embedding Method:
[Poincaré Embeddings for Learning Hierarchical Representations](https://papers.nips.cc/paper/2017/file/59dfa2df42d9e3d41f5b02bfc32229dd-Paper.pdf) & 
[Joint Learning of Hyperbolic Label Embeddings for Hierarchical Multi-label Classification: hierarchy embedding method](https://github.com/geoopt/geoopt/blob/master/examples/hyperbolic_multiclass_classification.ipynb)

Loss function for embedding a known hierarchy:
[Unit Ball Model for Embedding Hierarchical Structures in the Complex Hyperbolic Space](https://arxiv.org/pdf/2105.03966v3.pdf)

Plots and manifold computations:
[Geoopt library and usage example](https://github.com/geoopt/geoopt/blob/master/examples/hyperbolic_multiclass_classification.ipynb)

Protein sequence embedding:
[T5 model for protein sequences](https://huggingface.co/Rostlab/prot_t5_xl_half_uniref50-enc)

## Setup

In [1]:
import torch
torch.manual_seed(1)
torch.cuda.manual_seed(1)

import logging
logging.basicConfig(level=logging.DEBUG,filename='../../logs/cath.log')   

from tqdm import tqdm
from datetime import datetime

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../..'))

import numpy as np
import matplotlib.pyplot as plt

from transformers import T5Tokenizer, T5EncoderModel, AutoConfig

# from src.cath.datasets import LabelDataset
# from src.utils import add_geodesic_grid
# from src.utils import choice

from src.datasets import SequenceLabelDataset, SequenceDataset
from src.cath import CathLabelDataset
from src.losses import SequenceLabelLoss, SequenceLoss, Sequence01Loss
from src.models import LabelEmbedModel, Classifier

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using {}".format(device))


%matplotlib inline

Using cuda


## Data

In [2]:
label_dataset = CathLabelDataset(path='../../data/cath/labels.csv', no_disconnected_per_connected=10)
seq_train_dataset = SequenceDataset(label_dataset, path='../../data/cath/train.csv', )

trainloader = torch.utils.data.DataLoader(
    seq_train_dataset,
    # SequenceLabelDataset(seq_train_dataset, label_dataset),
    batch_size=2,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

In [3]:
seq_val_dataset = SequenceDataset(label_dataset, path='../../data/cath/val.csv')

valloader = torch.utils.data.DataLoader(
    seq_val_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=1,
    pin_memory=True
)

### Memory check

In [4]:
for i in range(torch.cuda.device_count()):
    t = torch.cuda.get_device_properties(i).total_memory
    r = torch.cuda.memory_reserved(i)
    a = torch.cuda.memory_allocated(i)
    f = r-a  # free inside reserved

    print(f'#{i}\n\ttotal {t/1024/1024/1024}\n\treserved {r/1024/1024/1024}\n\tallocated {a/1024/1024/1024}\n\tfree {f/1024/1024/1024}')

#0
	total 47.46234130859375
	reserved 0.0
	allocated 0.0
	free 0.0


## Models

In [40]:
# seq_encoder = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc") #.to(device)


EMBEDDING_DIM = 1

config = AutoConfig.from_pretrained("t5-base")
# config.d_ff = 128
# config.d_model = EMBEDDING_DIM
config.vocab_size = 128


EMBEDDING_DIM = config.d_model

seq_encoder = T5EncoderModel.from_pretrained("t5-base", config=config, ignore_mismatched_sizes=True)

print(seq_encoder.config)
seq_encoder = torch.nn.DataParallel(seq_encoder).to(device) # for training on multiple GPUs

tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

Some weights of the model checkpoint at t5-base were not used when initializing T5EncoderModel: ['decoder.block.6.layer.2.DenseReluDense.wo.weight', 'decoder.block.11.layer.1.EncDecAttention.k.weight', 'decoder.block.9.layer.1.layer_norm.weight', 'decoder.block.8.layer.1.layer_norm.weight', 'decoder.block.7.layer.1.EncDecAttention.q.weight', 'decoder.block.6.layer.1.EncDecAttention.o.weight', 'decoder.block.9.layer.0.SelfAttention.q.weight', 'decoder.block.10.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.2.layer.2.DenseReluDense.wo.weight', 'decoder.block.4.layer.2.DenseReluDense.wo.weight', 'decoder.block.10.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.6.layer.2.DenseReluDense.wi.weight', 'decoder.block.6.layer.1.EncDecAttention.v.weight', 'decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.7.layer.0.SelfAttention.q.weight', 'decoder.block.2.layer.1.EncDecAttention.q.

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "pre

In [41]:
label_embeddings = LabelEmbedModel(label_dataset.no_labels(), emb_dim=EMBEDDING_DIM, eye=True) #.to(device)
label_embeddings = torch.nn.DataParallel(label_embeddings).to(device) # for training on multiple GPUs

In [42]:
classifier = Classifier(seq_encoder, tokenizer, label_embeddings, label_dataset.assignable_labels, device)

### Memory check

In [8]:
for i in range(torch.cuda.device_count()):
    t = torch.cuda.get_device_properties(i).total_memory
    r = torch.cuda.memory_reserved(i)
    a = torch.cuda.memory_allocated(i)
    f = r-a  # free inside reserved

    print(f'#{i}\n\ttotal {t/1024/1024/1024}\n\treserved {r/1024/1024/1024}\n\tallocated {a/1024/1024/1024}\n\tfree {f/1024/1024/1024}')

#0
	total 47.46234130859375
	reserved 0.376953125
	allocated 0.3402829170227051
	free 0.03667020797729492


## Train

In [9]:
def evaluate(classifier, loader, loss_fn, process_batch):
    running_loss = 0
    
    classifier.train(False)
    
    for idx, batch in enumerate(loader):
        seqs, labels = process_batch(batch)
        
        labels = labels.to(device)
        
        logits = classifier(seqs)
        loss = loss_fn(logits, labels)
        
        running_loss += loss.item()
    
    return running_loss / len(loader)
    

In [ ]:
def train(classifier, trainloader, valloaders, validate_after=1000, batch_loss_after=50):
    running_loss = 0
    last_loss = 0
    
    pbar = tqdm(enumerate(trainloader))
    
    classifier.train(True)
        
    for idx, batch in pbar:
        seqs, _, labels_hot = batch #, edges = batch
        labels_hot = labels_hot.to(device)
        # edges = edges.to(device)
        
        optimizer.zero_grad()
        
        logits = classifier(seqs)
        # loss =  train_loss(logits, labels_hot, label_embeddings(edges))
        loss =  train_loss(logits, labels_hot)
        
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        # REPORT
        
        if idx % batch_loss_after == batch_loss_after - 1:
            last_loss = running_loss / batch_loss_after
            logging.debug(f"train set avg bce loss for batches {idx - batch_loss_after + 1} - {idx} / {len(trainloader)}: {last_loss}")
            pbar.set_description(f"train set avg bce loss for batches {idx - batch_loss_after + 1} - {idx} / {len(trainloader)}: {last_loss}")
            running_loss = 0.
        
        if idx % validate_after == 0:
            
            for name, valloader in valloaders.items():
                bce = evaluate(classifier, valloader, eval_bce_loss, lambda batch: (batch[0], batch[2]))
                l01 = evaluate(classifier, valloader, eval_01_loss, lambda batch: (batch[0], batch[1]))

                print(f'{name} bce loss: {bce}')
                print(f'{name} 0-1 loss: {l01}')

                logging.debug(f'{name} bce loss: {bce}')
                logging.debug(f'{name} 0-1 loss: {l01}')

            classifier.train(True)

In [ ]:
train_loss = SequenceLoss()
eval_bce_loss = SequenceLoss()
eval_01_loss = Sequence01Loss()

optimizer = torch.optim.Adam([
        {'params': classifier.parameters(), 'lr': 0.1}
        # {'params': seq_encoder.parameters(), 'lr': 0.001}
    ])

EPOCHS = 100
for epoch in range(EPOCHS):
    loss = train(classifier, trainloader, valloader, batch_loss_after=1000, validate_after=1000)

### Memory check

In [ ]:
for i in range(torch.cuda.device_count()):
    t = torch.cuda.get_device_properties(i).total_memory
    r = torch.cuda.memory_reserved(i)
    a = torch.cuda.memory_allocated(i)
    f = r-a  # free inside reserved

    print(f'#{i}\n\ttotal {t/1024/1024/1024}\n\treserved {r/1024/1024/1024}\n\tallocated {a/1024/1024/1024}\n\tfree {f/1024/1024/1024}')

## Playground

#### Overfitting

In [26]:
import pandas as pd

overfit_seq_df = pd.read_csv('../../data/cath/train.csv', sep=' ').sample(n=100)
overfit_dataset = SequenceDataset(label_dataset, df=overfit_seq_df)

overfit_loader = torch.utils.data.DataLoader(
    overfit_dataset,
    # SequenceLabelDataset(seq_train_dataset, label_dataset),
    batch_size=1,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

print('overfit loader size:', len(overfit_loader))
overfit_seq_df

,label,seq
0,7190,SHYEEGPGKNLPFSVENKWSLLAKMCLYFGSAFATPFLVVRHQLLKT
1,4932,KCSLEMIDHIVGNQPDQEMVSASEWYLKNLQFHRFWSVDDTQVHTE...
2,6989,ISRKWEKKNKIVYPPQLPGEPRRPAEIYHCRRQIKYSKDKMWYLAK...
3,5220,GFPVRPQVPLRPMTYKAALDISHFLKEKGGLEGLIWSQRRQEILDL...
4,7199,GAQVSTQKTGAHETSLSATGNSIIHYTNINYYKDAASNSANRQDFT...
...,...,...
95,6884,TKLLKKIINETAFAASTQESRPILTGVHFVLSQHKELKTVATDSHR...
96,3401,APTNYNLRVIEVTVAANALATRYSVALPSHKDNSNSERGNLRDFMD...
97,2298,MKHDYTNPPWNAKVPVQRAMQWMPISQKAGAAWGVDPQLITAIIAI...
98,5157,ESAVLRGFLILGKEDKRYGPALSINELSNLAKGEKANVLIGQGDVV...


In [44]:
train_loss = SequenceLoss()
eval_bce_loss = SequenceLoss()
eval_01_loss = Sequence01Loss()

optimizer = torch.optim.Adam([
        {'params': classifier.parameters(), 'lr': 0.1}
        # {'params': seq_encoder.parameters(), 'lr': 0.001}
    ])

EPOCHS = 100
for epoch in range(EPOCHS):
    loss = train(classifier, trainloader, {'val set': valloader, 'overfit set': overfit_loader}, validate_after=1000, batch_loss_after=100)
    print(f"----------- epoch {epoch} ----------")

0it [00:00, ?it/s]

val set bce loss: 108.45213092803955
val set 0-1 loss: 1.0


3it [00:27,  7.26s/it]

overfit set bce loss: 69.40430959817299
overfit set 0-1 loss: 0.97


avg loss batches 900 - 999 / 7683: 71.67213508799672: : 999it [01:35, 19.09it/s] 

...

val set bce loss: 1556.7828216552734
val set 0-1 loss: 1.0


avg loss batches 5900 - 5999 / 7683: 70.76242486727718: : 6003it [07:15,  2.04s/it]

overfit set bce loss: 63.31829859864925
overfit set 0-1 loss: 0.8


avg loss batches 6100 - 6199 / 7683: 62.4289705523569: : 6221it [07:25, 13.97it/s] 


KeyboardInterrupt: 

In [ ]:
evaluate(trainloader, eval_01_loss, lambda batch: (batch[0], batch[1]))

In [46]:
for seq, label, _ in overfit_loader:
    print(label, classifier.classify(seq))

tensor([2411]) tensor([2159], device='cuda:0')
tensor([5364]) tensor([1883], device='cuda:0')
tensor([3269]) tensor([3216], device='cuda:0')
tensor([1051]) tensor([1022], device='cuda:0')
tensor([5505]) tensor([1883], device='cuda:0')
tensor([3397]) tensor([3397], device='cuda:0')
tensor([779]) tensor([779], device='cuda:0')
tensor([4297]) tensor([1883], device='cuda:0')
tensor([3281]) tensor([1883], device='cuda:0')
tensor([3965]) tensor([1883], device='cuda:0')
tensor([2398]) tensor([2511], device='cuda:0')
tensor([4313]) tensor([1883], device='cuda:0')
tensor([2511]) tensor([2511], device='cuda:0')
tensor([1723]) tensor([1723], device='cuda:0')
tensor([5513]) tensor([1883], device='cuda:0')
tensor([3274]) tensor([3274], device='cuda:0')
tensor([3246]) tensor([1883], device='cuda:0')
tensor([4734]) tensor([1883], device='cuda:0')
tensor([783]) tensor([718], device='cuda:0')
tensor([2054]) tensor([2054], device='cuda:0')
tensor([855]) tensor([1883], device='cuda:0')
tensor([1834]) ten

In [47]:
label_dataset.assignable_labels[1833]

tensor(3353)